Caricate il data set Arrythmia in una DataFrame Pandas.

Effettuate la pulizia dei dati eliminando le colonne che contengono più di un '?'. Nelle colonne che contengono un solo '?' sostituitelo col valor medio della colonna.

Sviluppate classificatori k-NN per classificare i dati con uno split train/test 60/40 per diversi valori di k e determinate il migliore (in base all'accuracy).

Effettuate una riduzione di dimensionalità con PCA in modo da mantenere il 90% della varianza ed effettuate nuovamente la classificazione k-NN per il solo valore migliore di k trovato in precedenza.

Riportate infine la matrice di confusione dei classificatori col miglior valore di k sia nel caso a dimensionalità piena che a dimensionalità ridotta.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Caricate il data set Arrythmia in una DataFrame Pandas.
import pandas as pd
import numpy as np

# labels = range(280)
arr_df = pd.read_csv('/content/drive/MyDrive/ColabNb/arrhythmia/arrhythmia.data')
arr_df.shape

(451, 280)

In [ ]:
# Effettuate la pulizia dei dati eliminando le colonne che contengono più di un '?'.
# Nelle colonne che contengono un solo '?' sostituitelo col valor medio della colonna

guess_point_count = 0
indexes_to_remove = []
indexes_to_fix = []
for col in range(arr_df.shape[1]):
  guess_point_count = 0
  for row in range(arr_df.shape[0]):
    if (arr_df.iloc[row, col] == '?'):
      guess_point_count += 1
  if (guess_point_count > 1):
    indexes_to_remove.append(col)
  if (guess_point_count == 1):
    indexes_to_fix.append(col)


print(indexes_to_remove)
print(indexes_to_fix)

[10, 11, 13]
[12, 14]


In [ ]:
indexes_to_keep = set(range(280))
for x in indexes_to_remove:
  indexes_to_keep.remove(x)

len(list(indexes_to_keep))

filt_df = arr_df.iloc[:, list(indexes_to_keep)]
filt_df.shape

(451, 277)

In [ ]:
for col in range(filt_df.shape[1]):
  for row in range(filt_df.shape[0]):
    if (filt_df.iloc[row, col] == '?'):
      row_indexes_to_keep = set(range(filt_df.shape[0]))
      row_indexes_to_keep.remove(row)
      row_indexes_to_keep = list(row_indexes_to_keep)
      array_for_avg = [int(x) for x in list(filt_df.iloc[row_indexes_to_keep, col])]
      filt_df.iloc[row, col] = round(np.average(array_for_avg))

In [ ]:
dict_types = dict()
for col in range(filt_df.shape[1]):
  for row in range(filt_df.shape[0]):
    t = str(type(filt_df.iloc[row, col]))
    if (dict_types.get(t) != None):
      dict_types[t] += 1
    else:
      dict_types[t] = 1

dict_types

{"<class 'numpy.int64'>": 69905,
 "<class 'float'>": 902,
 "<class 'numpy.float64'>": 54120}

In [ ]:
# portiamo tutti i valori a float per evitare errori di tipo in seguito
for col in range(filt_df.shape[1]):
  for row in range(filt_df.shape[0]):
    filt_df.iloc[row, col] = float(filt_df.iloc[row, col])


In [ ]:
data = filt_df.iloc[:, range(filt_df.shape[1] - 1)].to_numpy(copy=True)
target = filt_df.iloc[:, -1].to_numpy(copy=True)
print(f'original shape: {filt_df.to_numpy().shape}')
print(data.shape, target.shape)
print()

original shape: (451, 277)
(451, 276) (451,)



In [ ]:
# Sviluppate classificatori k-NN per classificare i dati con uno split train/test 60/40
# per diversi valori di k e determinate il migliore (in base all'accuracy)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=323, train_size=0.60, test_size=0.40)

print(X_train.shape)
print(X_test.shape)

(270, 276)
(181, 276)


In [ ]:
# Costruisco il modello e ne valuto le performance
from sklearn.neighbors import KNeighborsClassifier

for k in range(1, 10):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X=X_train, y=y_train)
  # Stampo le performance
  print(f'{k}: {knn.score(X_test, y_test):.2%}')

1: 64.09%
2: 62.98%
3: 64.64%
4: 65.19%
5: 63.54%
6: 65.19%
7: 64.09%
8: 62.43%
9: 62.98%


In [ ]:
# Effettuate una riduzione di dimensionalità con PCA
# in modo da mantenere il 90% della varianza
# ed effettuate nuovamente la classificazione k-NN per il solo valore migliore di k trovato in precedenza.

# k = 4
import pandas as pd
from sklearn.decomposition import PCA

# pca = PCA(n_components=0.90)
pca = PCA(n_components=0.90, svd_solver='full')

data_n = pca.fit(data).transform(data)
print(data_n.shape)

(451, 27)


In [ ]:
from sklearn.model_selection import train_test_split

Xn_train, Xn_test, yn_train, yn_test = train_test_split(data_n, target, random_state=323, train_size=0.60, test_size=0.40)

print(Xn_train.shape)
print(Xn_test.shape)

(270, 27)
(181, 27)


In [ ]:
knn4 = KNeighborsClassifier(n_neighbors=4)
knn4.fit(X=Xn_train, y=yn_train)
print(f'Accuracy: {knn4.score(Xn_test, yn_test):.2%}')

Accuracy: 66.30%


In [ ]:
# Riportate infine la matrice di confusione dei classificatori col miglior valore di k
# sia nel caso a dimensionalità piena che a dimensionalità ridotta.

In [ ]:
from sklearn.metrics import confusion_matrix

# Calcolo la matrice di confusione del classificatore a dimensionalità piena per k=4
knn_cm = KNeighborsClassifier(n_neighbors=4)
knn_cm.fit(X=X_train, y=y_train)
predicted = knn_cm.predict(X=X_test)
expected = y_test
confusion = confusion_matrix(y_true=expected, y_pred=predicted)
confusion

array([[101,   0,   0,   1,   0,   1,   0,   0,   0,   0,   0,   0],
       [ 11,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  4,   1,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,   0,   0,   1,   1,   0,   0,   0,   0,   0,   0,   0],
       [  2,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0],
       [  8,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0],
       [ 14,   2,   0,   0,   0,   0,   0,   0,   6,   0,   0,   0],
       [  3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0],
       [ 10,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0]])

In [ ]:
# Calcolo la matrice di confusione del classificatore a dimensionalità ridotta per k=4
knn_cmn = KNeighborsClassifier(n_neighbors=4)
knn_cmn.fit(X=X_train, y=y_train)
predicted = knn_cmn.predict(X=X_test)
expected = y_test
confusion2 = confusion_matrix(y_true=expected, y_pred=predicted)
confusion2

array([[101,   0,   0,   1,   0,   1,   0,   0,   0,   0,   0,   0],
       [ 11,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  4,   1,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,   0,   0,   1,   1,   0,   0,   0,   0,   0,   0,   0],
       [  2,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0],
       [  8,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0],
       [ 14,   2,   0,   0,   0,   0,   0,   0,   6,   0,   0,   0],
       [  3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0],
       [ 10,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0]])

Nell'ambito del Machine learning, la matrice di confusione, detta anche tabella di errata classificazione, restituisce una rappresentazione dell'accuratezza di classificazione statistica.

Ogni colonna della matrice rappresenta i valori predetti, mentre ogni riga rappresenta i valori reali. L'elemento sulla riga i e sulla colonna j è il numero di casi in cui il classificatore ha classificato la classe "vera" i come classe j. Attraverso questa matrice è osservabile se vi è "confusione" nella classificazione di diverse classi.